In [1]:
import os
import json
import tkinter as tk
from tkinter import filedialog
import time
import concurrent.futures
import threading
import subprocess
import re

def extract_working_directory(netj_file):
    try:
        with open(netj_file, 'r') as file:
            data = json.load(file)
            return data.get('work_dir', '')
    except Exception as e:
        print(f"Error extracting working directory from {netj_file}: {str(e)}")
        return None

def monitor_and_update_active_file(working_directory, sormax, maxit):
    target_file = os.path.join(working_directory, "ACTIVE")
    while not os.path.exists(target_file):
        print(f"Waiting for Active file in {working_directory}...")
        time.sleep(10)
    time.sleep(15)
    print(f"Active file found in {working_directory}. Starting monitoring and updating...")

    # Monitor and update logic here
    with open(target_file, "r") as f:
        lines = f.readlines()

    for i, line in enumerate(lines):
        if "SORMAX" in line and "MAXIT" in line:
            names = line.split('!')[-1].strip().split(',')
            print(names)
            print(line)
            if "SORMAX" in names and "MAXIT" in names:

                splits = [0, 11, 18, 25, 35]
                values = [line[splits[i]:splits[i+1]] for i in range(len(splits)-1)]
                values = [x.strip() for x in values]
                print(values)
                sormax_index = names.index('SORMAX')
                maxit_index = names.index('MAXIT')
                values[sormax_index] = format(float(sormax), '.4f')
                values[maxit_index] = maxit
                print(values)

                formatted_line = "     {:}     {:}   {:}      {:}   !{}".format(values[0], values[1], values [2], values[3], ','.join(names))
                lines[i] = formatted_line + '\n'

    with open(target_file, "w") as f:
        f.writelines(lines)

    print(f"Active file updated and monitoring completed in {working_directory}. Exiting monitoring.")
    return

def run_simulation(netj_file, sormax, maxit):
    regex = "\d{4}"
    netj_num = re.findall(regex, netj_file)[1]
    if not (working_directory := extract_working_directory(netj_file)):
        return
    os.chdir(working_directory)
    cfd_time_file = os.path.join(working_directory, 'CFD_TIME')

    if not os.path.exists(cfd_time_file):
        with open(cfd_time_file, 'w') as f:
            f.write('0')

    print(f"Running simulation for {netj_file} in {working_directory}")

    rainbow_dir = os.path.join(os.getenv('localappdata'), 'Programs', 'ReactionEngineeringInternational', 'CFS-EPA', 'cowtip', 'x64', 'Release',)
    os.chdir(rainbow_dir)

    # Run RainbowBatch.exe with the .NETJ file as an argument
    # process = subprocess.Popen(['RainbowBatch.exe', netj_file])
    with subprocess.Popen(['RainbowBatch.exe', netj_file], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text = 'TRUE') as process:
        for line in iter(process.stdout.readline, ''):
            print(netj_num, ':', line, end='')
        return_code = process.wait()
        if return_code != 0:
            print(f"RainbowBatch.exe {netj_file} failed with return code {return_code}.")

    #Monitor for the presence of reks35.out
    while True:
        if os.path.exists(os.path.join(working_directory, 'streamline_pp.vtk')):
            time.sleep(30)
            process.terminate()
            process.wait()

            # print('Simulation completed.')
            return
        else:
            time.sleep(10)

def run_sim_and_monitor(netj_file, sormax, maxit):
    # Create a thread for running the simulation
    sim_thread = threading.Thread(target=run_simulation, args=(netj_file, sormax, maxit))
    monitor_thread = threading.Thread(target=monitor_and_update_active_file, args=(extract_working_directory(netj_file), sormax, maxit))

    # Start the simulation thread
    sim_thread.start()
    monitor_thread.start()
    
    # Wait for the simulation thread to finish
    sim_thread.join()
    monitor_thread.join()
    # print('sim and monitor threads joined')

def main():
    root = tk.Tk()
    root.withdraw()

    netj_files = filedialog.askopenfilenames(
        title="Select .NETJ files",
        filetypes=[("NETJ Files", "*.netj")],
        initialdir='C:/Users/KWILKES/Desktop/PFAS_Modeling/cfs_para_test_cond/0000_docs',
        multiple=True
    )

    sormax = input("Please enter a value for SORMAX (default: 2.5): ") or "2.5"
    maxit = input("Please enter a value for MAXIT (default: 500): ") or "500"

    max_concurrent_processes = 4

    with concurrent.futures.ThreadPoolExecutor(max_concurrent_processes) as executor:
        futures = []
        for netj_file in netj_files:
            # Remove ACTIVE file if it exists
            working_directory = extract_working_directory(netj_file)
            active_file = os.path.join(working_directory, "ACTIVE")
            if os.path.exists(active_file):
                os.remove(active_file)
                
            # Use the new wrapper function to ensure both simulation and monitoring are run together
            future = executor.submit(run_sim_and_monitor, netj_file, sormax, maxit)
            futures.append(future)
            # print('futures appended')
        concurrent.futures.wait(futures)
        # print('concurrent futures wait done')

    print("All simulations and monitoring/updating completed.")
    root.destroy()
if __name__ == "__main__":
    main()

Waiting for Active file in C:\Users\KWILKES\Desktop\PFAS_Modeling\cfs_para_test_cond\001_1stream\1120_41kW_CF4_port1...
Running simulation for C:/Users/KWILKES/Desktop/PFAS_Modeling/cfs_para_test_cond/001_1stream/0000_docs/1120_rainbow_41kW_CF4_port1.netj in C:\Users\KWILKES\Desktop\PFAS_Modeling\cfs_para_test_cond\001_1stream\1120_41kW_CF4_port1
1120 : loading document C:/Users/KWILKES/Desktop/PFAS_Modeling/cfs_para_test_cond/001_1stream/0000_docs/1120_rainbow_41kW_CF4_port1.netj...
1120 : loading json-type net file C:/Users/KWILKES/Desktop/PFAS_Modeling/cfs_para_test_cond/001_1stream/0000_docs/1120_rainbow_41kW_CF4_port1.netj
1120 : executing...
1120 :  IPAS      5936800
1120 :          122          93          65
1120 :  BEWARE OF SOME CHANGE IN DATA FILE !!
1120 :  CHECK SUBROUTINE SETUP TO MAKE SURE YOU GOT 
1120 :  RIGHT DATA FILE FORMAT!!!
1120 :                                                                                
1120 :       <<EPA Lab Kiln primary chamber>>         